실습문제:
단톡방에 주어진 URL은 쿠팡에서 "노트북" 키워드로 검색된
상품 목록에 대한 주소이다.
https://www.coupang.com/np/search?q=%EB%85%B8%ED%8A%B8%EB%B6%81&listSize=72&channel=user

이 주소의 웹 페이지로부터
상품의 이름, 가격, 할인율, 할인가, 별점, 리뷰수, 상품이미지의 경로를
수집하여 엑셀 파일로 저장하시오.

제한시간: 4:30분~ 5시15분 (45분)

In [1]:
# 필요한모듈참조
import requests
    # request는 온라인 상의 데이터를 수집한다
    # request로 갖고 온 데이터값이 JSON형태이면 dictionary로 변환하면 된다
    # request로 갖고 온 데이터값이 HTML + CSS 이면 BeautifulSoup라는 클래스를 통해 HTML + CSS 안에 있는 내용들을 추출해준다
from bs4 import BeautifulSoup
from pandas import DataFrame
    # pandas는 통계패키지; DataFrame은 표 형태로 출력해줌

In [3]:
# 수집할 컨텐츠가 있는 웹 페이지의 주소
url = "https://www.coupang.com/np/search?q=%EB%85%B8%ED%8A%B8%EB%B6%81&listSize=72&channel=user"
# 브라우저 버전 정보
userAgent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36"

# 접속객체 생성
session = requests.Session()

# 접속객체에 부가정보(header) 삽입
session.headers.update({
    "Referer": "",
    "User-Agent": userAgent,
    "Accept-Language": "ko-KR,ko;q=0.8,en-US;q=0.5,en;q=0.3"
})


# 생성한 접속객체를 활용하여 API에 접속
r = session.get(url)

# 접속에 실패한 경우
if r.status_code != 200:
        # .status_code = 400 혹은 403 -> 잘못된 접근, 권한 없음; 404 -> Not Found, 보통 url주소를 잘못 적음
        # .status_code = 500번대 -> Server Error, 내가 할 수 있는 것 없음 (서버에 문제가 있는 것)
        # .status_code = 200 -> okay
    # 에러코드와 에러메시지 출력
    msg = "[%d Error] %s 에러가 발생합" % (r.status_code, r.reason)
    # 에러를 강제로 생성시킴
    raise Exception(msg)

# 인코딩 형식 지정하여 beautifulsoup 객체를 생성
r.encoding = "utf-8"
    # 가져온 내용 r에 인코딩을 지정하고
soup = BeautifulSoup(r.text)
    # BeautifulSoup한테 갖고 온 내용을 던져주면 분석 가능한(추출할 수 있는) 형태로 객체화 시켜줌
soup
    # 이 soup을 가지고 내가 원하는 값을 출력/추출하면 됨


<!DOCTYPE html>

<!--[if lte IE 7 ]><html class="lt-ie9 lt-ie8" lang="ko-KR"><![endif]-->
<!--[if IE 8 ]><html class="lt-ie9" lang="ko-KR"><![endif]-->
<!--[if (gte IE 9)|!(IE)]><!--><html lang="ko-KR"><!--<![endif]-->
<head>
<meta charset="utf-8"/>
<meta content="zaNrGtrOLMjglkziY2IvmL8dOXyCWHGArDHqFazJQVI" name="google-site-verification"/>
<meta content="on" http-equiv="x-dns-prefetch-control"/>
<link href="//cart.coupang.com" rel="dns-prefetch"/>
<link href="//assets.coupang.com" rel="dns-prefetch"/>
<link href="//assets2.coupang.com" rel="dns-prefetch"/>
<link href="//assets.coupangcdn.com" rel="dns-prefetch"/>
<link href="//asset1.coupangcdn.com" rel="dns-prefetch"/>
<link href="//private.coupang.com" rel="dns-prefetch"/>
<link href="//img1a.coupangcdn.com" rel="dns-prefetch"/>
<link href="//image1.coupangcdn.com" rel="dns-prefetch"/>
<link href="//thumbnail1.coupangcdn.com" rel="dns-prefetch"/>
<link href="//static.coupangcdn.com" rel="dns-prefetch"/>
<link href="//www.facebook.

In [4]:
# 필요한 내용 추출
product = soup.select(".search-product-wrap")
print("검색 결과 수: ", len(product))
# product

검색 결과 수:  72


In [10]:
# 추출할 데이터를 병합할 빈 리스트
data = []

for p in product:
    # 이름 추출
    nameEl = p.select(".name")
    if nameEl:
        name = nameEl[0].text.strip()
    else:
        name = None
    # print(name)

    # 가격 추출
    basePriceEl = p.select(".base-price")
    if basePriceEl:
        basePrice = basePriceEl[0].text.strip().replace(',', '')
        # 여기서 값이 ','를 포함하고 있기 때문에 .replace()를 통해 ','을 없애줘야 된다 (통계할 떄 ',' 있으면 곤란함)
    else:
        basePrice = None
    # print(basePrice)

    # 할인율 추출
    discountRateEl = p.select(".instant-discount-rate")
    if discountRateEl:
        discountRate = int(discountRateEl[0].text.strip().replace("%", ""))/100
        # 여기서 값이 '%'을 포함하고 있기 때문에 .replace()를 통해 '%'를 없애줘야 된다 (통계할 떄 '%' 있으면 곤란함)
    else: 
        discountRate = None
    # print(discountRate)

    # 할인가 추출
    priceValueEl = p.select(".price-value")
    if priceValueEl:
        priceValue = int(priceValueEl[0].text.strip().replace(',', ''))
        # 여기서 값이 '%'을 포함하고 있기 때문에 .replace()를 통해 '%'를 없애줘야 된다 (통계할 떄 '%' 있으면 곤란함)
    else: 
        priceValue = None
    # print(priceValue)

    # 별점 추출
    ratingEl = p.select(".rating")
    if ratingEl:
        rating = float(ratingEl[0].text.strip())
    else:
        rating = None
    # print(rating)

    # 리뷰수 추출
    ratingTotalCountEl = p.select(".rating-total-count")
    if ratingTotalCountEl:
        ratingTotalCount = ratingTotalCountEl[0].text.strip()
        ratingTotalCount = ratingTotalCount[1:-1]
    else:
        ratingTotalCount = None
    # print(ratingTotalCount)

    # 상품이미지 주소 추출
    searchProductWrapImgEl = p.select(".search-product-wrap-img")
    if searchProductWrapImgEl:
        if "data-img-src" in searchProductWrapImgEl[0].attrs:
            image ="https:"+ searchProductWrapImgEl[0].attrs['data-img-src']
        else:
            image ="https:"+ searchProductWrapImgEl[0].attrs['src']
    else:
        image = None
    # print(image)

    # 중고상품판별조건 
        # 중고 상품은 가격 정보가 없다
        # 중고상품의 '최저가'가 crawling으로 수집이 안 됨
        # 그러므로 중고 상품 걸러내기
    if not basePrice and not priceValue:
        continue

    # 추출한 내용을 딕셔너리로 병합
    item = {
        "이름": name, 
        "가격": basePrice, 
        "할인율": discountRate, 
        "할인가": priceValue, 
        "별점": discountRate, 
        "리뷰수": ratingTotalCount, 
        "상품이미지": image
        }
    data.append(item)

data

[{'이름': '베이직스 2023 베이직북S 10.1, 화이트, 셀러론, 128GB, 8GB, WIN11 Pro, BB1022FW',
  '가격': None,
  '할인율': None,
  '할인가': 398000,
  '별점': None,
  '리뷰수': '14',
  '상품이미지': 'https://thumbnail8.coupangcdn.com/thumbnails/remote/230x230ex/image/retail/images/2023/04/07/10/2/ff136b6a-25f6-4eae-9efa-68bc90d49fb1.jpg'},
 {'이름': '레노버 2022 아이디어패드 노트북, 아틱그레이, SLIM3 15ITL6, 코어i7, 256GB, 16GB, Free DOS',
  '가격': None,
  '할인율': None,
  '할인가': 629000,
  '별점': None,
  '리뷰수': '362',
  '상품이미지': 'https://thumbnail8.coupangcdn.com/thumbnails/remote/230x230ex/image/retail/images/2023/02/27/16/6/cbd90a5e-8fa6-41bb-aef8-6bf853a9529a.png'},
 {'이름': '베이직스 2022 베이직북 14 3세대, BB1422SS, 256GB, 화이트, WIN11 Pro, 셀러론, 8GB',
  '가격': None,
  '할인율': None,
  '할인가': 398000,
  '별점': None,
  '리뷰수': '1142',
  '상품이미지': 'https://thumbnail9.coupangcdn.com/thumbnails/remote/230x230ex/image/retail/images/3205128823935169-9b961481-e75f-4737-87e0-40d36d9d5e98.png'},
 {'이름': '삼성전자 2021 노트북 플러스2 15.6, NT550XDA-K24AT, WIN11 Pro, 8GB, 256GB, 펜티엄,

In [11]:
# df = DataFrame(data)
# df.to_excel("연습문제_쿠팡검색결과.xlsx")
# df